In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:

from audio.redis import *
from audio.audio import *
#from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

import json
import time
pd.set_option('display.max_colwidth', None)

In [4]:

async def get_next_chunk_start(redis_inner_client, audio_name, length,shift):
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    diarization_result = diarization.data

    if len(diarization_result)>0:
        last_speech = diarization_result[-1]

        ended_silence = length - last_speech['end']
        print(ended_silence)
        if ended_silence<2:
            print('interrupted')
            return last_speech['start']+shift
        

        else:
            print('non-interrupted') 
            return last_speech['end']+shift



    else: return None



async def diarize(client_id, audio_name, shift, redis_inner_client):
    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    done = await redis_inner_client.brpop(f'DiarizeReady:{audio_name}')
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    return diarization.data

    


async def transcribe(audio_name, redis_inner_client):
    await redis_inner_client.lpush('Audio2TranscribeQueue', f'{audio_name}:{client_id}')
    _,done = await redis_inner_client.brpop(f'TranscribeReady:{audio_name}')
    transcription =  Transcript(audio_name,redis_inner_client)
    await transcription.get()
    return transcription.data


In [5]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'
import subprocess
from pydub import AudioSegment
import io
from time import sleep

from audio.redis import Transcript

In [6]:
redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [7]:
await get_connections_df('initialFeed_audio',redis_stream_client)

,queue,length


In [8]:
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_ids = [c.replace('initialFeed_audio:','') for c in connections]


connections =  await get_connections('initialFeed_audio',redis_stream_client)

In [9]:
#[await redis_stream_client.delete(c) for c in connections]

In [10]:
connection_id = '7c781102-9ee6-4f3e-af7b-54ca79adb82f' #'david_audio'#

In [11]:

path = f'/app/testdata/{connection_id}.webm'

In [12]:
def expand_over_timeindex(df,shift,column):
    if len(df)>0:
        expanded_df = pd.DataFrame(columns=[column, 'time'])
        for index, row in df.iterrows():
            time_range = np.arange(row['start'] * 100, row['end'] * 100, 1).astype(int)

            temp_df = pd.DataFrame({
                column: row[column],
                'time': time_range
            })
            expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)
        expanded_df['conv_time'] = (expanded_df['time'] + shift*100).astype(int)

        return expanded_df.sort_values("time")

In [13]:
on_start = False

In [14]:

#on_start = True

In [15]:

step = 60
max_length=60

In [16]:
start = 0

In [17]:
redis_stream_client = await get_stream_redis()
redis_inner_client = await get_inner_redis()

In [18]:
diarizations = []
transcriptions = []

In [19]:
def clean_diarization_result(diarization_result,min_len=0.75):
    df = pd.DataFrame(diarization_result)
    df['speaker'] = np.where(df['end']-df['start']<min_len,np.nan,df['speaker'])
    return df.fillna(method='ffill').fillna(method='bfill').to_dict(orient='records')

In [20]:
speaker_dict = {'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc':'David',
                 'd59b85f5-9fdf-4d71-9a4e-54c8c3caded0':'me'}

In [23]:
rlen=0

In [24]:
await writestream2file(connection_id,redis_stream_client)
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,start+max_length)

slice_duration = audio_slicer.audio.duration_seconds
print(slice_duration)

if slice_duration > step:

    audio_data = await audio_slicer.export_data()
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()

    diarization_result, transcription_result = await asyncio.gather(
            diarize(client_id, audio_name, start,redis_inner_client),
            transcribe(audio_name,redis_inner_client)
        )
    
if len(diarization_result)>0:
    diarization_result = clean_diarization_result(diarization_result)


    diarization_df = expand_over_timeindex(pd.DataFrame(diarization_result),start,'speaker')
    diarization_df['shift'] = start
    transcription_df = expand_over_timeindex(pd.concat([pd.DataFrame(t) for t in transcription_result]),start,'word')
    transcription_df['shift'] = start
    diarizations.append(diarization_df)
    transcriptions.append(transcription_df)
    start_ = await get_next_chunk_start(redis_inner_client, audio_name, slice_duration,start)
    start = start_ if start_ else start+slice_duration
    diarization_df = pd.concat(diarizations).sort_values('shift',ascending=False).drop_duplicates('conv_time').sort_values('conv_time')
    transcription_df = pd.concat(transcriptions).sort_values('shift',ascending=False).drop_duplicates('conv_time').sort_values('conv_time')
    df = diarization_df[['conv_time',"speaker"]].set_index('conv_time').join(transcription_df[['conv_time','word']].set_index('conv_time')).reset_index().dropna()
    df['cleaned_text'] = df['word'].str.replace('[^a-zA-Z]', '', regex=True)
    df['word_switch'] = df['cleaned_text']!=df['cleaned_text'].shift()
    df = df[df['word_switch']].drop(columns = ['word_switch','cleaned_text'])
    df['speaker_switch'] = df['speaker']!=df['speaker'].shift()
    df['speaker_switch'] = df['speaker_switch'].cumsum()
    df = df.reset_index(drop=True)
    speakers = df.drop_duplicates('speaker_switch').set_index('speaker_switch')['speaker'].to_frame('speaker')
    r = (df.groupby('speaker_switch')['word'].apply(lambda x:''.join(x)).to_frame('text')
    .join(speakers)
    .join(df.groupby('speaker_switch')['conv_time'].min().to_frame('min'))
    .join(df.groupby('speaker_switch')['conv_time'].max().to_frame('max')))
    r['span'] = r['max']-r['min']
    r['speaker'] = r['speaker'].replace(speaker_dict)
    if rlen==len(r):max_length+=step
    max_length = 20
    rlen==len(r)
else:
    max_length+=step
r.tail(5)

60.336
None
-0.07995925297114326
interrupted


/tmp/ipykernel_1770324/506151659.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='ffill').fillna(method='bfill').to_dict(orient='records')


,text,speaker,min,max,span
speaker_switch,,,,,
1,"one has to do with the carbohydrates and restriction of carbohydrates and some people feeling healthier when they cut carbohydrates up. My guess is, this is my theory to be tested, that people feel better cutting carbohydrates out because the diet that we eat in the United States and in industrialized countries, the carbohydrates are largely crap processed. They're It's their process. It's like starch, simple sugar. It's things that contribute to glycemic index. It's these sugars that we eat. They make it to our small intestine. They get chopped up into simple sugars, absorbed into our bloodstream, and we have a ton of glucose then coursing through our veins, which we know is bad and can lead to things like diabetes. If the carbohydrates that were in our diet were complex carbohydrates, dietary fiber, and we like to refer to the subset of dietary fiber that the microbiota can actually access as microbiota-accessible carbohydrates. The reason that we like that term is it has the word carbohydrate in it. It's to point out that not all carbs are bad. It's just there are bad carbs or carbs that are bad if you consume them in too high a quantity, things like table sugar and simple starches, but there are good carbs as well. These microbiota-accessible carbohydrates are the complex ones that we can't digest. Yes.",0c85f6fa-f9ad-4450-a302-0e6098ec2fa7,1445,10031,8586


In [386]:
start

160.0848896434635

In [387]:
max_length

20

In [297]:
r.groupby('speaker')['span'].sum().sort_values()

speaker
5062be38-0ba9-4e87-94e3-e3365454f932       10
f1a39f08-6e76-4662-b714-d0fa1e561a06       27
24cdf2f8-b3a8-4791-bbc7-39fb4509acd7       98
edf6d12a-1984-4292-bb69-58b1b6dfed17      157
365365a4-6a7c-46e9-9626-aab4998dda4a      216
708c5594-68e1-44ce-a2a1-2fdbcad350c0      444
069a3a37-3ff4-45bf-bf7f-280b0b926428      933
525c6198-3697-410f-b0b4-638704df615d     1869
me                                       9777
David                                   73259
Name: span, dtype: int64

In [300]:
path = f'/app/testdata/{connection_id}.webm'
from audio.audio import *
from pathlib import Path
audio_slicer_ =await AudioSlicer.from_file(path,'webm')

In [303]:
audio_slicer_.slice(957,967)

In [304]:
n = 28
slice_ = [r.loc[n]['min']/100,r.loc[n]['max']/100]
audio_slicer_.slice(slice_[0],slice_[1])

In [81]:
r['speaker'].unique()

array(['c91b86b5-e41c-48b3-9a81-2addd33cbc08',
       '361db38b-6a25-48db-b5f3-c04812002c14',
       '73279877-dcb8-4194-8716-d149db6a94e4',
       'c9822d44-6764-40d0-82a2-57cc3530fc9f',
       'c0549776-4c0b-4e55-bf95-d196ea4a56ec',
       'ad6acaad-ac6a-4583-817c-d29d2559c608',
       'c1fa9976-c12d-4ad4-80f7-8e14b0e58c22',
       '2443495f-a52a-46a9-be44-0a1d8678ddd3',
       '258a6be3-80af-4fa7-be2f-0037e526a25f'], dtype=object)